In [43]:
import pandas as pd, numpy as np
np.set_printoptions(suppress=True)# 关掉科学计数法
import glob
import os
import csv
# 一次性merge多个pct_chg
from functools import reduce
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels import regression

# import tushare as ts
import time, urllib
# ts.set_token('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')
# pro = ts.pro_api('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')

# import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)


# from plotly.graph_objs import Scatter,Layout
# import plotly
# import plotly.offline as py
# import numpy as np
# import plotly.graph_objs as go

# #setting offilne
# plotly.offline.init_notebook_mode(connected=True)

# TechnicalIndicators

In [44]:
class TechnicalIndicators(object):

    def EMA(df, n, price_col): # n = 5
        """
        Exponential Moving Average
        rationale CHECKED, code CHECKED, updated.

        params:
            df: pd dataframe
            n: number of days = 5
        """
        EMA = df[price_col].ewm(span=n, min_periods=n - 1).mean().rename('EMA_' + str(n))
        return EMA

    def OBV(df, n, price_col, vol_col): # n = 5
        """On-balance Volume

        On Balance Volume (OBV) measures buying and selling pressure as a cumulative indicator that adds 
        volume on up days and subtracts volume on down days. OBV was developed by Joe Granville and introduced 
        in his 1963 book, Granville's New Key to Stock Market Profits. It was one of the first indicators to 
        measure positive and negative volume flow. Chartists can look for divergences between OBV and price 
        to predict price movements or use OBV to confirm price trends.

        http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:on_balance_volume_obv
        """
        df = df.reset_index()
        i = 0
        OBV = [0]
        while i < df.index[-1]:
            if df.at[i + 1, price_col] - df.at[i, price_col] > 0:
                OBV.append(df.at[i + 1, vol_col])
            if df.at[i + 1, price_col] - df.at[i, price_col] == 0:
                OBV.append(0)
            if df.at[i + 1, price_col] - df.at[i, price_col] < 0:
                OBV.append(-df.at[i + 1, vol_col])
            i = i + 1
        OBV = pd.Series(OBV)
        OBV_ma = pd.Series(OBV.rolling(window=n).mean(), name = 'OBV_' + str(n))
        return OBV_ma

    # Rationale checked
    def MFI(df, n, hi_col, lo_col, price_col, vol_col): # n = 14
        """Money Flow Index and Ratio, updated.
        http://stockcharts.com/docs/doku.php?id=scans:indicators#money_flow_index_mfi

        """
        df = df.reset_index()
        PP = (df[hi_col] + df[lo_col] + df[price_col]) / 3
        i  = 0
        PosMF = [0]
        while i < df.index[-1]:
            if PP[i + 1] > PP[i]:
                PosMF.append(PP[i + 1] * df.at[i + 1, vol_col])
            else:
                PosMF.append(0)
            i = i + 1
        PosMF = pd.Series(PosMF)
        TotMF = PP * df[vol_col]
        MFR   = pd.Series(PosMF / TotMF)
        MFI   = pd.Series(MFR.rolling(window = n, center = False).mean(), name = 'MFI_' + str(n))
        df    = df.join(MFI).set_index("index")
        return df["MFI_" + str(n)]

    # Done
    # Rationale checked
    def RSI(df, n, hi_col, lo_col): # n = 14
        """
        Relative Strength Index, updated.
        Conventional parameters: n = 14, 0.3 and 0.7 are two conventional thresholds
        """
        df = df.reset_index()
        i = 0
        UpI = [0]
        DoI = [0]
        while i + 1 <= df.index[-1]:
            UpMove = df.at[i + 1, hi_col] - df.at[i, hi_col]
            DoMove = df.at[i, lo_col] - df.at[i + 1, lo_col]
            if UpMove > DoMove and UpMove > 0:
                UpD = UpMove
            else: UpD = 0
            UpI.append(UpD)
            if DoMove > UpMove and DoMove > 0:
                DoD = DoMove
            else: DoD = 0
            DoI.append(DoD)
            i = i + 1
        UpI   = pd.Series(UpI)
        DoI   = pd.Series(DoI)
        PosDI = UpI.ewm(span = n, min_periods = n - 1).mean()
        NegDI = DoI.ewm(span = n, min_periods = n - 1).mean()
        RSI   = pd.Series(PosDI / (PosDI + NegDI), name = 'RSI_' + str(n))
        df    = df.join(RSI).set_index("index")
        return df["RSI_" + str(n)]

    def BIAS(df, n, price_col):
        BIAS = df[price_col]-df[price_col].rolling(window=n).mean().rename('BIAS_'+str(n))
        return BIAS

    def MACD(df, n_fast, n_slow, n_macd, price_col): # n_fast = 12, n_slow = 26
        """
        http://stockcharts.com/docs/doku.php?id=scans:indicators
        MACD, MACD Signal and MACD difference, rationale CHECKED, code CHECKED, updated
        # Conventional look-back window for calculating MACDsign is 9
        """
        EMAfast = df[price_col].ewm(span = n_fast, min_periods = n_fast - 1).mean()
        EMAslow = df[price_col].ewm(span = n_slow, min_periods = n_slow - 1).mean()
        MACD = pd.Series(EMAfast - EMAslow, name = 'MACD_' + str(n_fast) + '_' + str(n_slow))
        MACDsign = MACD.ewm(span = n_macd, min_periods = n_macd-1).mean().rename('MACDsign_' + str(n_fast) + '_' + str(n_slow))
        MACDdiff = pd.Series(MACD - MACDsign, name = 'MACDdiff_' + str(n_fast) + '_' + str(n_slow))
        df['MACD_Diff'] = MACD
        df['MACD_Diff_EMA'] = MACDsign
        df['MACD'] = MACDdiff
        df['SIGNAL_STATUS'] = df['MACD'].apply(lambda x: "多头状态" if x>0 else ("空头状态" if x<0 else "无信号状态"))
        return df

# TimeSeriesToolbox

In [45]:
class GetData(object):
    
    def get_date_price_code_df(path, ticker_list, date_col, price_col, code_col):
        # for etf data cols are 'date', 'close', 'code'
        ticker_df_list = []
#         print(ticker_list)
        for ticker in ticker_list:
            print(ticker)
            try:
#                 print("get thru")
                
                ticker_df = pd.read_csv(path+ticker+".csv")
                ticker_df[code_col] = ticker_df[code_col].astype(str)
                ticker_df = ticker_df.sort_values(date_col)
                ticker_df = ticker_df[[date_col, price_col, code_col]]
#                 print(ticker_df)
                ticker_df_list.append(ticker_df)
            except Exception as e:
                print(e)
        try:
            tickers_data_concated = pd.concat(ticker_df_list)
            tickers_data_concated.reset_index(inplace=True)
            del tickers_data_concated['index']  
        except Exception as e:
            print(e)
#         print(tickers_data_concated)
        return tickers_data_concated

In [46]:
class TimeSeriesToolbox(object):
        
    def make_numeric_signals(series):
        for item in series:
            if item =="多":
                return 1
            elif item =="空":
                return -1
            else:
                return 0 
            
    def merge_weights_and_signal(df_actions,
                                 df_wts,
                                 path,
                                 code_col,
                                 date_col,
                                 price_col,
                                 tgt_wts_mutiplier,
                                 account_value):
        if df_actions.empty:
            print("There's no data in df_actions. No actional signals for today!")
            pass
            
        else:
            # 合并仓位数据和信号数据
            df_actions_with_weights = df_wts.merge(df_actions, on =code_col)

            # 仓位太小，创建2倍仓位信息, e.g. tgt_wts_mutiplier = 2
            df_actions_with_weights['weight_enlarged'] = df_actions_with_weights['weight']*tgt_wts_mutiplier
    #         print(df_actions_with_weights)
            # 提取下一日要操作的tickers
            tickers = list(df_actions_with_weights[code_col])
    #         print(tickers)
            # 得到这些tickers的收盘价数据
    #         print(tickers)
            tickers_closes = GetData.get_date_price_code_df(path,
                                                             tickers,
                                                             date_col, 
                                                             price_col, 
                                                             code_col)
    #         print(tickers_closes)
            # 创建今日date信息
            last_date = tickers_closes[date_col].values[-1]
    #         print(last_date)
            # 提取最近一天的tickers的收盘价数据
            tickers_closes_last_date = tickers_closes[tickers_closes[date_col] == last_date]

            # 创建最终的信号-仓位指示信息
            df_actions_with_weights = df_actions_with_weights.merge(tickers_closes_last_date, on = [date_col,
                                                                                                    code_col])
            df_actions_with_weights['tgt_shares'] = account_value*\
                                                    df_actions_with_weights['weight_enlarged']/\
                                                    df_actions_with_weights[price_col]
            return df_actions_with_weights
     
    def merge_current_pos_with_target_pos(path, cur_positions, tgt_last_macd_signals):
        tgt_last_macd_signals['TYPE'] = "TARGET"
        # the following variables should be assigned first
        cur_pos_macd = MACDSignals(path, 
                        cur_positions, 
                        date_col, 
                        code_col, 
                        price_col, 
                        n_fast, 
                        n_slow, 
                        n_macd, 
                        ticker_type)
        cur_pos_macd_signals, \
        cur_pos_last_macd_signals, \
        cur_pos_df_actions = cur_pos_macd.calc_macd_signals()
        cur_pos_last_macd_signals['TYPE'] = 'CUR_POS'
        tgt_cur_macd_signal_df = cur_pos_last_macd_signals.merge(tgt_last_macd_signals, on = [date_col,code_col], how = 'outer')
        return tgt_cur_macd_signal_df

# PlotToolBox

In [47]:
class PlotToolbox(object):
    
    def pie_graph(values, labels, pie_length, pie_width, title_name):
        # draw pie graph
        plt.figure(1, figsize = (pie_length, pie_width))
        plt.axes(aspect=1)
        plt.pie(x=values, labels=labels, autopct='%3.1f %%')
        plt.title(title_name, fontsize = 15)
        plt.show()
        
        
    def plot_macd_signals(ticker, macd_signals, tail_num):
        ticker_macd_signals = macd_signals[macd_signals[code_col]==ticker]
        ticker_macd_signals.set_index(date_col, inplace = True)
        ticker_macd_signals[[code_col,"MACD"]].tail(tail_num).plot(figsize = (15,6))

In [48]:
# 在分析环境里，筛选出tickers，然后使用MACD_signals

class MACDSignals(object):
    
    def __init__(self, stocks_path, tickers, date_col, code_col, price_col, n_fast, n_slow, n_macd, ticker_type):
        self.path = stocks_path
        self.tickers = tickers
        self.date_col = date_col
        self.code_col = code_col
        self.price_col = price_col
        self.n_fast = n_fast
        self.n_slow = n_slow
        self.n_macd = n_macd
        self.ticker_type = ticker_type
        self.mkt_data = self.get_mkt_data_df()

    def get_mkt_data_df(self):
    # e.g. ch_db_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
        csv_path = self.path+"*.csv"
        files = glob.glob(csv_path)
        ticker_df_list = []
        for ticker in tickers:
            try:
                ticker_df = pd.read_csv(self.path+ticker+".csv")
                ticker_df[self.code_col] = ticker_df[self.code_col].astype(str)
                ticker_df = ticker_df.sort_values(self.date_col)
                ticker_df_list.append(ticker_df)
            except Exception as e:
                print(e)
        try:
            tickers_data_concated = pd.concat(ticker_df_list)
            tickers_data_concated.reset_index(inplace=True)
            del tickers_data_concated['index']  
        except Exception as e:
            print(e)
        return tickers_data_concated
    

    def calc_macd_signals(self):
        tickers_data_concated = self.mkt_data
#         print(tickers_data_concated)
        signal_record = []
        signal_data = []
        if len(self.tickers)!=1:
            for ticker in self.tickers:
                try:
                    if self.ticker_type == "float":
                        single_ticker_df = tickers_data_concated[tickers_data_concated[self.code_col]==float(ticker)]
                    elif self.ticker_type == "string":
                        single_ticker_df = tickers_data_concated[tickers_data_concated[self.code_col]==ticker]
                        
                    signal_df = TechnicalIndicators.MACD(single_ticker_df, self.n_fast, self.n_slow, self.n_macd, self.price_col)
                    
                    signal_data.append(signal_df)
                except:
                    pass
            signal_data_df = pd.concat(signal_data)
        else:
            try:                
                signal_df = TechnicalIndicators.MACD(single_ticker_df, self.n_fast, self.n_slow, self.n_macd, self.price_col)
            except:
                pass
            signal_data_df = signal_df

        # v1 is the version of generating the og macd signals
        signal_data_df['SIGNAL_DIRECTION'] = signal_data_df['SIGNAL_STATUS'].apply(lambda x: TimeSeriesToolbox.make_numeric_signals(x))
        print("signal_data_df", signal_data_df.head(3))
        signal_data_df['SIGNAL_DIRECTION_DIFF'] = signal_data_df.groupby([self.code_col])['SIGNAL_DIRECTION'].diff()
        signal_data_df['SIGNAL_ACTION'] = signal_data_df['SIGNAL_DIRECTION_DIFF'].apply(lambda x: "LONG" if x==2 else("SHORT" if x==-2 else "NO CHANGE"))
#         print(signal_data_df)
        most_recent_signals = signal_data_df.groupby([self.code_col])[[self.date_col,self.code_col,'SIGNAL_STATUS','SIGNAL_ACTION']].tail(1)
        df_actions = most_recent_signals[most_recent_signals["SIGNAL_ACTION"]!="NO CHANGE"]
        return signal_data_df, most_recent_signals, df_actions

# Risk Parity

In [49]:
from scipy.optimize import minimize

class RiskParity(object):
    
    def __init__(self, stocks_path, tickers, 
                 date_col, code_col, price_col, 
                 ticker_type, asset_name, draw_pie_graph):
        
        self.path = stocks_path
        self.tickers = tickers
        self.date_col = date_col
        self.code_col = code_col
        self.price_col = price_col
        self.ticker_type = ticker_type
        self.asset_name = asset_name
        self.draw_pie_graph = draw_pie_graph
        self.ticker_df_list = self.get_date_price_code_return_list()
        self.tgt_returns = self.ticker_df_list
        self.tgt_merged_returns = self.merge_dfs_by_ticker(self.tgt_returns, 
                                                           self.date_col)
        self.wts, self.risk = self.get_smart_weight(self.tgt_merged_returns, 
                                                    method='risk parity', 
                                                    cov_adjusted=False, 
                                                    wts_adjusted=False)
        self.df_wts, self.risk_parity_tickers, self.weights = self.get_df_wts()
        
        
    # Get date_col, price_col, code_col, pct_chg_col
    def get_date_price_code_return_list(self):
        # for etf data cols are 'date', 'close', 'code'
        ticker_df_list = []
        for ticker in self.tickers:
            try:
                ticker_df = pd.read_csv(self.path+ticker+".csv")
                ticker_df = ticker_df.sort_values(self.date_col)
                ticker_df = ticker_df[[self.date_col, 
                                       self.price_col, 
                                       self.code_col]]
                ticker_df['pct_chg'] = ticker_df[self.price_col].pct_change()
                ticker_df = ticker_df[[self.date_col, 'pct_chg']].dropna()
                ticker_df.columns = [self.date_col, ticker]
                ticker_df_list.append(ticker_df)
            except Exception as e:
                print(e)
        return ticker_df_list
    
    
    def merge_dfs_by_ticker(self, ticker_df_list, date_col):
        merged_all = reduce(lambda left, right: pd.merge(left, right, on=date_col), ticker_df_list)
#         merged_all = reduce(merge_df_for_reduce, ticker_df_list)
        merged_all.set_index(self.date_col, inplace=True)
        merged_all.dropna(how="all", axis = 1, inplace = True)
        merged_all.fillna(method="ffill", inplace = True)
        return merged_all
        
        
    def get_smart_weight(self, pct, method, cov_adjusted, wts_adjusted):
        if cov_adjusted == False:
            #协方差矩阵
            cov_mat = pct.cov()
        else:
            #调整后的半衰协方差矩阵
            cov_mat = pct.iloc[:len(pct)/4].cov()*(1/10.) + pct.iloc[len(pct)/4+1:len(pct)/2].cov()*(2/10.) +\
                pct.iloc[len(pct)/2+1:len(pct)/4*3].cov()*(3/10.) + pct.iloc[len(pct)/4*3+1:].cov()*(4/10.)
        if not isinstance(cov_mat, pd.DataFrame):
            raise ValueError('cov_mat should be pandas DataFrame！')

        omega = np.matrix(cov_mat.values)  # 协方差矩阵
        
        a, b = np.linalg.eig(np.array(cov_mat)) #a为特征值,b为特征向量
        a = np.matrix(a)
        b = np.matrix(b)
        # 定义目标函数
    
        def fun1(x):
            tmp = (omega * np.matrix(x).T).A1
            risk = x * tmp/ np.sqrt(np.matrix(x) * omega * np.matrix(x).T).A1[0]
            delta_risk = [sum((i - risk)**2) for i in risk]
            return sum(delta_risk)

        def fun2(x):
            tmp = (b**(-1) * omega * np.matrix(x).T).A1
            risk = (b**(-1)*np.matrix(x).T).A1 * tmp/ np.sqrt(np.matrix(x) * omega * np.matrix(x).T).A1[0]
            delta_risk = [sum((i - risk)**2) for i in risk]
            return sum(delta_risk)
    
        # 初始值 + 约束条件 
        x0 = np.ones(omega.shape[0]) / omega.shape[0]  
        bnds = tuple((0,None) for x in x0)
        cons = ({'type':'eq', 'fun': lambda x: sum(x) - 1})
        options={'disp':False, 'maxiter':1000, 'ftol':1e-20}
        
        
        #------------------问题出在这里------------------
        if method == 'risk parity':
            res = minimize(fun1, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)        
        elif method == 'pc risk parity':
            res = minimize(fun2, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
        #------------------------------------
        
        else:
            raise ValueError('method error！！！')
            
        # 权重调整
        if res['success'] == False:
            # print res['message']
            pass
        wts = pd.Series(index=cov_mat.index, data=res['x'])

        if wts_adjusted == True:
            wts[wts < 0.0001]=0.0
            wts = wts / wts.sum()
        elif wts_adjusted == False:
            wts = wts / wts.sum()
        else:
            raise ValueError('wts_adjusted should be True/False！')

        risk = pd.Series(wts * (omega * np.matrix(wts).T).A1 / np.sqrt(np.matrix(wts) * omega * np.matrix(wts).T).A1[0],index = cov_mat.index)
        risk[risk<0.0] = 0.0
        return wts,risk
    
        
    def get_df_wts(self):
        df_wts = pd.DataFrame(self.wts)
        df_wts.reset_index(inplace = True)
        df_wts.columns = [self.asset_name, 'weight']
        risk_parity_tickers = list(df_wts[self.asset_name])
        weights = list(df_wts['weight'])
        return df_wts, risk_parity_tickers, weights


In [50]:
# Mannually check our current position holdings for potential exit signals:
cur_positions = [
#     "510180",#180ETF
#     "510300",#300ETF
#     "510810",#上海国企
#     "510850",#工银上50
#     "510880",#红利ETF
#     '512000', #券商ETF -
#     '512010',#医药ETF
#     "512400",#有色ETF
#     "512660",#军工ETF
#     '512290',#生物医药ETF -   
    "512380", #MSCI中国指数
    '512690',#酒ETF
#     '512800',#银行ETF
#     '512880',#证券ETF -
     '512980', #传媒ETF
    '159928',#消费ETF

#     '513050',#中概互联网
#     '513100',#纳指ETF
#     '518880',#黄金ETF
#     "159905",#深红利
    "159910", #深100ETF
#     "159920",#恒生ETF
    "159928", #消费ETF
#     "159959", #央企ETF -
#     "159939"# 信息技术
    '159938',#医药 -
    '159952', #创业ETF
#     ''#券商ETF
#     '512960',#央调ETF
#     ''#证券ETF
    '513500'#标普500
#     "513100"#纳指
    
]
len(cur_positions)

9

# Calculating MACD Signals

In [51]:
if __name__=="__main__":
    stocks_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
    etfs_df = pd.read_csv("filtered_50_etfs_by_vol_20200224.csv")
    tickers = list(etfs_df['etf'].str.split(".",expand = True)[0])
#     date_col = 'trade_date'
#     code_col = 'ts_code'
#     price_col = 'close'
    date_col = 'date'
    code_col = 'code'
    price_col = 'close'
    ticker_type = 'string'
    asset_name = "code"
    tgt_wts_mutiplier = 4
    account_value = 100000
    
#     # ========For calculating MACD signals========
    n_fast = 12
    n_slow = 26
    n_macd = 9
    macd = MACDSignals(stocks_path, 
                        tickers, 
                        date_col, 
                        code_col, 
                        price_col, 
                        n_fast, 
                        n_slow, 
                        n_macd, 
                        ticker_type)
    macd_signals, last_macd_signals, df_actions = macd.calc_macd_signals()

/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

signal_data_df          date   open  close   high    low     volume    code  MACD_Diff  \
0  2013-07-29  2.633  2.626  2.640  2.614  1404335.0  518880        NaN   
1  2013-07-30  2.632  2.646  2.648  2.632   929931.0  518880        NaN   
2  2013-07-31  2.643  2.657  2.659  2.640   699761.0  518880        NaN   

   MACD_Diff_EMA  MACD SIGNAL_STATUS  SIGNAL_DIRECTION  
0            NaN   NaN         无信号状态                 0  
1            NaN   NaN         无信号状态                 0  
2            NaN   NaN         无信号状态                 0  


# Delete tickers that are not up-to-date to make sure RP can function well

In [52]:
last_date = last_macd_signals['date'].values[-1]
tickers = list(last_macd_signals[last_macd_signals['date']==last_date]['code'])
print("The number of Tickers Left is: ", len(tickers))

The number of Tickers Left is:  49


# Calculate Risk-Parity Positions

In [53]:
# ========For calculating risk-parity weights========
risk_parity = RiskParity(stocks_path,
                          tickers,
                          date_col,
                          code_col,
                          price_col,
                          ticker_type,
                          asset_name,
                          True)

df_wts, risk_parity_tickers, weights = risk_parity.get_df_wts()

# Check if there's any new position to be entered

In [54]:
df_actions_with_weights = TimeSeriesToolbox.merge_weights_and_signal(df_actions,
                                                                 df_wts,
                                                                 stocks_path,
                                                                 code_col,
                                                                 date_col,
                                                                 price_col,
                                                                 tgt_wts_mutiplier,
                                                                 account_value)
#     PlotToolbox.pie_graph(weights, risk_parity_tickers, 8, 8, "Risk Parity Allocation")

510180
510050
510850


In [55]:
last_macd_signals

,date,code,SIGNAL_STATUS,SIGNAL_ACTION
1732,2020-09-03,518880,空头状态,NO CHANGE
3351,2020-09-03,513500,多头状态,NO CHANGE
5134,2020-09-03,513100,多头状态,NO CHANGE
7051,2020-09-03,159920,空头状态,NO CHANGE
8870,2020-09-03,510500,空头状态,NO CHANGE
9754,2020-09-03,513050,多头状态,NO CHANGE
12347,2020-09-03,510180,空头状态,SHORT
14263,2020-09-03,510900,空头状态,NO CHANGE
16856,2020-09-03,159901,多头状态,NO CHANGE
17175,2020-09-03,512290,空头状态,NO CHANGE


In [56]:
last_macd_signals[last_macd_signals['SIGNAL_STATUS']=='多头状态']

,date,code,SIGNAL_STATUS,SIGNAL_ACTION
3351,2020-09-03,513500,多头状态,NO CHANGE
5134,2020-09-03,513100,多头状态,NO CHANGE
9754,2020-09-03,513050,多头状态,NO CHANGE
16856,2020-09-03,159901,多头状态,NO CHANGE
18849,2020-09-03,512010,多头状态,NO CHANGE
20547,2020-09-03,159928,多头状态,NO CHANGE
25039,2020-09-03,159919,多头状态,NO CHANGE
25368,2020-09-03,512690,多头状态,NO CHANGE
30703,2020-09-03,159938,多头状态,NO CHANGE
32819,2020/8/7,159915,多头状态,NO CHANGE


In [57]:
last_macd_signals[last_macd_signals['SIGNAL_STATUS']=='多头状态'][last_macd_signals['SIGNAL_ACTION']=='LONG']

/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,date,code,SIGNAL_STATUS,SIGNAL_ACTION


# Merge current position and target positions

In [58]:
tgt_cur_pos_df = TimeSeriesToolbox.merge_current_pos_with_target_pos(stocks_path, cur_positions, last_macd_signals)

signal_data_df              date   open  close   high    low    volume    code  MACD_Diff  \
63527  2019-04-30  0.960  0.973  0.980  0.960  176255.0  512380        NaN   
63528  2019-05-06  0.944  0.912  0.944  0.902  254559.0  512380        NaN   
63529  2019-05-07  0.911  0.923  0.929  0.907  108649.0  512380        NaN   

       MACD_Diff_EMA  MACD SIGNAL_STATUS  SIGNAL_DIRECTION  
63527            NaN   NaN         无信号状态                 0  
63528            NaN   NaN         无信号状态                 0  
63529            NaN   NaN         无信号状态                 0  


/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

ValueError: cannot reindex from a duplicate axis

# Re-entry list can be separately found from tgt_cur_pos_df

In [59]:
def find_recent_goldcross_close(close, signal_diff):
    if signal_diff == 1 or signal_diff == 2:
        return close

    
    
macd_signals['recent_goldcross'] = macd_signals.apply(lambda row: find_recent_goldcross_close(row['close'],row['SIGNAL_DIRECTION_DIFF']), axis=1)
# forward-fill all NaNs for the recent_goldcross to pave way for calculating the pct_diff between current 
# close and the rencent goldcross close
macd_signals['recent_goldcross_ffill'] = macd_signals.groupby("code")['recent_goldcross'].apply(lambda x: x.fillna(method="ffill"))
macd_signals['pctchg_from_recent_goldcrossclose'] = (macd_signals['close']-macd_signals['recent_goldcross_ffill'])/macd_signals['recent_goldcross_ffill']
macd_signals['reentry_rank'] = macd_signals.groupby(["date"])['pctchg_from_recent_goldcrossclose'].rank(ascending = True)
last_day = macd_signals['date'].values[-1]
macd_signals_reentry = macd_signals[macd_signals['date']==last_day][macd_signals['pctchg_from_recent_goldcrossclose']<0]

/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [60]:
all_reentry_tickers = list(macd_signals_reentry['code'])
tickers_to_reenter = []
for ticker in all_reentry_tickers:
    if ticker not in cur_positions:
        tickers_to_reenter.append(ticker)

In [61]:
tickers_to_reenter_df = pd.DataFrame(tickers_to_reenter, columns=['code'])
df_actions_with_weights_reentry = df_wts.merge(tickers_to_reenter_df, on = ['code'])
macd_signals_reentry = macd_signals[macd_signals['date']==last_day]
df_actions_with_weights_reentry = df_actions_with_weights_reentry.merge(macd_signals_reentry, on = 'code')
df_actions_with_weights_reentry['weight_enlarged'] = df_actions_with_weights_reentry['weight']*tgt_wts_mutiplier

In [62]:
df_actions_with_weights_reentry['tgt_shares'] = account_value*\
                                                    df_actions_with_weights_reentry['weight_enlarged']/\
                                                    df_actions_with_weights_reentry[price_col]

df_actions_with_weights_reentry = df_actions_with_weights_reentry[['code','date','SIGNAL_STATUS','SIGNAL_ACTION',\
                                                                   'weight_enlarged','close','tgt_shares',\
                                                                   'pctchg_from_recent_goldcrossclose','reentry_rank']]
df_actions_with_weights_reentry.sort_values('reentry_rank', ascending=True, inplace=True)
df_actions_with_weights_reentry_L = df_actions_with_weights_reentry[df_actions_with_weights_reentry['SIGNAL_STATUS']=="多头状态"]

In [63]:
df_actions_with_weights_reentry_L

,code,date,SIGNAL_STATUS,SIGNAL_ACTION,weight_enlarged,close,tgt_shares,pctchg_from_recent_goldcrossclose,reentry_rank
13,512330,2020-09-03,多头状态,NO CHANGE,0.050932,1.230,4140.851735,-0.014423,9.0
3,159901,2020-09-03,多头状态,NO CHANGE,0.059440,7.060,841.921265,-0.008427,10.0
23,512180,2020-09-03,多头状态,NO CHANGE,0.070673,1.392,5077.060550,-0.007133,12.0
22,510360,2020-09-03,多头状态,NO CHANGE,0.069353,1.619,4283.662949,-0.005528,14.0
20,512990,2020-09-03,多头状态,NO CHANGE,0.072148,1.620,4453.576627,-0.005525,15.0
9,159919,2020-09-03,多头状态,NO CHANGE,0.071832,4.968,1445.892053,-0.005007,16.0
17,510300,2020-09-03,多头状态,NO CHANGE,0.070527,4.884,1444.031785,-0.004890,18.0
18,510310,2020-09-03,多头状态,NO CHANGE,0.069478,2.183,3182.682850,-0.004560,20.0
16,510330,2020-09-03,多头状态,NO CHANGE,0.070594,4.903,1439.820649,-0.004467,21.0
5,512010,2020-09-03,多头状态,NO CHANGE,0.078812,2.901,2716.705411,-0.004461,22.0


In [64]:
selected_reentry_tickers = list(df_actions_with_weights_reentry_L['code'])
print("Tickers that need to be re-entered: ", selected_reentry_tickers)

Tickers that need to be re-entered:  ['512330', '159901', '512180', '510360', '512990', '159919', '510300', '510310', '510330', '512010', '159905', '159949']


# Check current positions & new positions

In [65]:
# Show if there are any actions needed to be made for our current positions
tgt_cur_pos_df[tgt_cur_pos_df['TYPE_x']=="CUR_POS"]

NameError: name 'tgt_cur_pos_df' is not defined

In [66]:
# Check if there are any new positions to be entered
tgt_cur_pos_df[(tgt_cur_pos_df['TYPE_x']!="CUR_POS")&(tgt_cur_pos_df['SIGNAL_ACTION_y']!="NO CHANGE")]

NameError: name 'tgt_cur_pos_df' is not defined

In [67]:
try:
    df_actions_with_weights = df_actions_with_weights[df_actions_with_weights['SIGNAL_ACTION']=='LONG']
except:
    df_actions_with_weights = []

try:
    df_actions_tickers = list(df_actions_with_weights['code'])
except:
    df_actions_tickers = []

symbol_list_to_backtest = df_actions_tickers+selected_reentry_tickers
print("All tickers that to be entered before backtesting are:", symbol_list_to_backtest)

All tickers that to be entered before backtesting are: ['512330', '159901', '512180', '510360', '512990', '159919', '510300', '510310', '510330', '512010', '159905', '159949']


# Backtest and Filter:

In [68]:
from CH_backtest import *

In [69]:
def macd_new_long_tickers_backup(last_macd_signals):
    long_tickers_df = last_macd_signals[last_macd_signals['SIGNAL_STATUS']=='多头状态'][last_macd_signals['SIGNAL_ACTION']=='LONG']
    long_tickers = list(long_tickers_df['code'])
    return long_tickers

In [70]:
def combine_macd_new_and_reentry_tickers_backup(last_macd_signals, selected_reentry_tickers):
    long_tickers = macd_new_long_tickers_backup(last_macd_signals)
    symbol_list_to_backtest_backup = long_tickers+selected_reentry_tickers
    symbol_list_to_backtest_backup = list(set(symbol_list_to_backtest_backup))
    return symbol_list_to_backtest_backup

In [71]:
symbol_list_to_backtest_backup = combine_macd_new_and_reentry_tickers_backup(last_macd_signals, selected_reentry_tickers)

/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [72]:
symbol_list_to_backtest_backup

['510310',
 '510300',
 '510330',
 '159949',
 '512180',
 '159901',
 '159905',
 '512990',
 '510360',
 '512330',
 '159919',
 '512010']

In [73]:
ticker_performances = {}

back_up = True

if back_up == True:
    symbol_list_to_backtest = symbol_list_to_backtest_backup
else:
    symbol_list_to_backtest = symbol_list_to_backtest

if __name__ == "__main__":   
    for ticker in symbol_list_to_backtest:
    # csv_dir = REPLACE_WITH_YOUR_CSV_DIR_HERE
        equity_folder = "./"
        csv_dir = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
#         data_cols = ['trade_date', 'open', 'high','low', 'close', 'volume','ts_code'] #for ETFs # 要改代码
#         data_cols = ['ts_code','trade_date','open','high','low','close','pre_close','change','pct_chg','vol','amount'] #for stocks
#         commissions = 5 # RMB/USD per trade #要改代码
        initial_capital = 1000000.0
        start_date = datetime.datetime(1991,12,1,0,0,0)
        start_date_str = str(start_date)
        heartbeat = 0.0
#         price_col = "close" #要改代码
#         qty = 5000 # 要改代码
        backtest = Backtest(csv_dir, 
                            [ticker], 
                            initial_capital, 
                            heartbeat,
                            start_date,
                            HistoricCSVDataHandler, 
                            SimulatedExecutionHandler, 
                            Portfolio, 
    #                         MovingAverageCrossStrategy,
                           MovingAverageConvergenceDivergence)

        backtest.simulate_trading()
        df_equity = pd.read_csv(ticker+"_performance"+".csv")
        df_equity.drop_duplicates("datetime", inplace = True)
        df_equity =df_equity[df_equity['datetime']>start_date_str]
        df_equity.index = df_equity['datetime']
        df_equity = df_equity[df_equity['total'].map(lambda x: str(x)!="nan")]
        df_equity.columns = ['datetime', 'market_value', 'cash', 'commission', 'total', 'returns',
        'equity_curve', 'drawdown']
        df_equity_copy = df_equity.copy()
        data = df_equity_copy
#         win_rate, mean_win_loss_ratio, bt_score, profits = performance(data)
        single_stats = performance(data)
#         print("TICKER: ", ticker)
        ticker_performances[ticker] = single_stats

Number of assets in the portfolio：  1
Position Value for each asset:  1000000.0
Asset:  510310 symbol_value:  0.9790000000000001 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.9670000000000001 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.986 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.998 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.8590000000000001 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.861 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.89 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.9 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.927 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.95 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.977 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.96 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.937 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.955 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.883 mkt_quantity:  5000
Asset:  510310 symbol_value:  0.87 mkt_quantit

Asset:  510310 symbol_value:  1.7180000000000002 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.719 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.837 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.75 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.771 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.6280000000000001 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.68 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.717 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.735 mkt_quantity:  5000
Asset:  510310 symbol_value:  1.761 mkt_quantity:  5000
Asset:  510310 symbol_value:  2.047 mkt_quantity:  5000
Asset:  510310 symbol_value:  2.126 mkt_quantity:  5000
Asset:  510310 symbol_value:  2.137 mkt_quantity:  5000
Asset:  510310 symbol_value:  2.178 mkt_quantity:  5000
Creating summary stats...
*****performance.py: 1. create_sharpe_ratio*****
*****performance.py: 2. create_drawdowns*****
Creating equity curve...
             510310       cash  commission    

Holding the position:  2014-01-29
Holding the position:  2014-01-30
Holding the position:  2014-02-07
Holding the position:  2014-02-10
Holding the position:  2014-02-11
Holding the position:  2014-02-12
Holding the position:  2014-02-13
Holding the position:  2014-02-14
Holding the position:  2014-02-17
Holding the position:  2014-02-18
Holding the position:  2014-02-19
Holding the position:  2014-02-20
Holding the position:  2014-02-21
Holding the position:  2014-02-24
Closed the position:  2014-02-25
This trade's pnl: -75.0
-------------------
Position is empty:  2014-02-25
Position is empty:  2014-02-26
Position is empty:  2014-02-27
Position is empty:  2014-02-28
Position is empty:  2014-03-03
Position is empty:  2014-03-04
Position is empty:  2014-03-05
Position is empty:  2014-03-06
Position is empty:  2014-03-07
Position is empty:  2014-03-10
Position is empty:  2014-03-11
Position is empty:  2014-03-12
Position is empty:  2014-03-13
Position is empty:  2014-03-14
Position is e

Holding the position:  2015-01-14
Closed the position:  2015-01-15
This trade's pnl: 50.0
-------------------
Position is empty:  2015-01-15
Position is empty:  2015-01-16
Position is empty:  2015-01-19
Position is empty:  2015-01-20
Position is empty:  2015-01-21
Position is empty:  2015-01-22
Position is empty:  2015-01-23
Position is empty:  2015-01-26
-------------------
Opened the position:  2015-01-27
Closed the position:  2015-01-28
This trade's pnl: -115.0
-------------------
Position is empty:  2015-01-28
-------------------
Opened the position:  2015-01-29
Closed the position:  2015-01-30
This trade's pnl: -165.0
-------------------
Position is empty:  2015-01-30
Position is empty:  2015-02-02
Position is empty:  2015-02-03
Position is empty:  2015-02-04
Position is empty:  2015-02-05
Position is empty:  2015-02-06
Position is empty:  2015-02-09
Position is empty:  2015-02-10
Position is empty:  2015-02-11
Position is empty:  2015-02-12
Position is empty:  2015-02-13
--------

Holding the position:  2016-03-25
Holding the position:  2016-03-28
Holding the position:  2016-03-29
Holding the position:  2016-03-30
Holding the position:  2016-03-31
Holding the position:  2016-04-01
Holding the position:  2016-04-05
Closed the position:  2016-04-06
This trade's pnl: 325.0
-------------------
Position is empty:  2016-04-06
Position is empty:  2016-04-07
-------------------
Opened the position:  2016-04-08
Holding the position:  2016-04-11
Holding the position:  2016-04-12
Holding the position:  2016-04-13
Closed the position:  2016-04-14
This trade's pnl: -20.0
-------------------
Position is empty:  2016-04-14
-------------------
Opened the position:  2016-04-15
Holding the position:  2016-04-18
Holding the position:  2016-04-19
Holding the position:  2016-04-20
Closed the position:  2016-04-21
This trade's pnl: -345.0
-------------------
Position is empty:  2016-04-21
Position is empty:  2016-04-22
Position is empty:  2016-04-25
Position is empty:  2016-04-26
Pos

...

Holding the position:  2015-09-18
Holding the position:  2015-09-21
Holding the position:  2015-09-22
Holding the position:  2015-09-23
Holding the position:  2015-09-24
Holding the position:  2015-09-25
Holding the position:  2015-09-28
Holding the position:  2015-09-29
Holding the position:  2015-09-30
Closed the position:  2015-10-08
This trade's pnl: -355.0
-------------------
Position is empty:  2015-10-08
Position is empty:  2015-10-09
-------------------
Opened the position:  2015-10-12
Holding the position:  2015-10-13
Holding the position:  2015-10-14
Holding the position:  2015-10-15
Holding the position:  2015-10-16
Holding the position:  2015-10-19
Holding the position:  2015-10-20
Holding the position:  2015-10-21
Holding the position:  2015-10-22
Holding the position:  2015-10-23
Holding the position:  2015-10-26
Holding the position:  2015-10-27
Holding the position:  2015-10-28
Holding the position:  2015-10-29
Holding the position:  2015-10-30
Holding the position:  20

Position is empty:  2016-08-08
Position is empty:  2016-08-09
Position is empty:  2016-08-10
Position is empty:  2016-08-11
-------------------
Opened the position:  2016-08-12
Holding the position:  2016-08-15
Holding the position:  2016-08-16
Holding the position:  2016-08-17
Holding the position:  2016-08-18
Holding the position:  2016-08-19
Holding the position:  2016-08-22
Holding the position:  2016-08-23
Holding the position:  2016-08-24
Holding the position:  2016-08-25
Holding the position:  2016-08-26
Holding the position:  2016-08-29
Closed the position:  2016-08-30
This trade's pnl: -60.0
-------------------
Position is empty:  2016-08-30
-------------------
Opened the position:  2016-08-31
Holding the position:  2016-09-01
Holding the position:  2016-09-02
Holding the position:  2016-09-05
Holding the position:  2016-09-06
Holding the position:  2016-09-07
Holding the position:  2016-09-08
Closed the position:  2016-09-09
This trade's pnl: -30.0
-------------------
Positio

Holding the position:  2017-09-04
Holding the position:  2017-09-05
Holding the position:  2017-09-06
Holding the position:  2017-09-07
Holding the position:  2017-09-08
Holding the position:  2017-09-11
Closed the position:  2017-09-12
This trade's pnl: -35.0
-------------------
Position is empty:  2017-09-12
Position is empty:  2017-09-13
Position is empty:  2017-09-14
Position is empty:  2017-09-15
Position is empty:  2017-09-18
-------------------
Opened the position:  2017-09-19
Holding the position:  2017-09-20
Holding the position:  2017-09-21
Holding the position:  2017-09-22
Holding the position:  2017-09-25
Holding the position:  2017-09-26
Holding the position:  2017-09-27
Holding the position:  2017-09-28
Holding the position:  2017-09-29
Holding the position:  2017-10-09
Holding the position:  2017-10-10
Holding the position:  2017-10-11
Holding the position:  2017-10-12
Holding the position:  2017-10-13
Holding the position:  2017-10-16
Holding the position:  2017-10-17
H

Position is empty:  2018-12-24
Position is empty:  2018-12-25
Position is empty:  2018-12-26
Position is empty:  2018-12-27
Position is empty:  2018-12-28
Position is empty:  2019-01-02
Position is empty:  2019-01-03
Position is empty:  2019-01-04
Position is empty:  2019-01-07
Position is empty:  2019-01-08
Position is empty:  2019-01-09
-------------------
Opened the position:  2019-01-10
Holding the position:  2019-01-11
Holding the position:  2019-01-14
Holding the position:  2019-01-15
Holding the position:  2019-01-16
Holding the position:  2019-01-17
Holding the position:  2019-01-18
Holding the position:  2019-01-21
Holding the position:  2019-01-22
Holding the position:  2019-01-23
Holding the position:  2019-01-24
Holding the position:  2019-01-25
Holding the position:  2019-01-28
Holding the position:  2019-01-29
Holding the position:  2019-01-30
Holding the position:  2019-01-31
Holding the position:  2019-02-01
Holding the position:  2019-02-11
Holding the position:  2019-

Holding the position:  2020-04-16
Holding the position:  2020-04-17
Holding the position:  2020-04-20
Holding the position:  2020-04-21
Holding the position:  2020-04-22
Closed the position:  2020-04-23
This trade's pnl: 700.0
-------------------
Position is empty:  2020-04-23
-------------------
Opened the position:  2020-04-24
Closed the position:  2020-04-27
This trade's pnl: -155.0
-------------------
Position is empty:  2020-04-27
Position is empty:  2020-04-28
Position is empty:  2020-04-29
Position is empty:  2020-04-30
Position is empty:  2020-05-06
Position is empty:  2020-05-07
-------------------
Opened the position:  2020-05-08
Holding the position:  2020-05-11
Holding the position:  2020-05-12
Holding the position:  2020-05-13
Holding the position:  2020-05-14
Holding the position:  2020-05-15
Holding the position:  2020-05-18
Holding the position:  2020-05-19
Holding the position:  2020-05-20
Holding the position:  2020-05-21
Holding the position:  2020-05-22
Holding the 

# Final Results

In [74]:
pd.DataFrame(ticker_performances)

,510310,510300,510330,159949,512180,159901,159905,512990,510360,512330,159919,512010
total_commission,785,855,785,435,255,1275,1055,595,565,555,885,805
gross_profit,7385,16270,14250,1065,1900,16335,3765,3365,3035,250,17415,5910
net_profit,6600,15415,13465,630,1645,15060,2710,2770,2470,-305,16530,5105
commission_impact,10.63%,5.26%,5.51%,40.85%,13.42%,7.81%,28.02%,17.68%,18.62%,222.0%,5.08%,13.62%
win %,44.87,44.71,48.72,34.88,48,34.65,34.29,42.37,48.21,32.73,47.73,33.75
PL Ratio,2.25,2.38,1.74,2.23,2.34,2.8,2.3,2.23,1.78,1.97,2.13,2.96
overall_score,0.46,0.51,0.33,0.13,0.6,0.32,0.13,0.37,0.34,-0.03,0.49,0.34


In [75]:
df_actions_with_weights

,code,weight,date,SIGNAL_STATUS,SIGNAL_ACTION,weight_enlarged,close,tgt_shares


In [76]:
df_actions_with_weights_reentry_L

,code,date,SIGNAL_STATUS,SIGNAL_ACTION,weight_enlarged,close,tgt_shares,pctchg_from_recent_goldcrossclose,reentry_rank
13,512330,2020-09-03,多头状态,NO CHANGE,0.050932,1.230,4140.851735,-0.014423,9.0
3,159901,2020-09-03,多头状态,NO CHANGE,0.059440,7.060,841.921265,-0.008427,10.0
23,512180,2020-09-03,多头状态,NO CHANGE,0.070673,1.392,5077.060550,-0.007133,12.0
22,510360,2020-09-03,多头状态,NO CHANGE,0.069353,1.619,4283.662949,-0.005528,14.0
20,512990,2020-09-03,多头状态,NO CHANGE,0.072148,1.620,4453.576627,-0.005525,15.0
9,159919,2020-09-03,多头状态,NO CHANGE,0.071832,4.968,1445.892053,-0.005007,16.0
17,510300,2020-09-03,多头状态,NO CHANGE,0.070527,4.884,1444.031785,-0.004890,18.0
18,510310,2020-09-03,多头状态,NO CHANGE,0.069478,2.183,3182.682850,-0.004560,20.0
16,510330,2020-09-03,多头状态,NO CHANGE,0.070594,4.903,1439.820649,-0.004467,21.0
5,512010,2020-09-03,多头状态,NO CHANGE,0.078812,2.901,2716.705411,-0.004461,22.0


# Visualize Signal Status on Long/Shorts

In [ ]:
def MACD_long_short_counts(macd_signals, date_col, code_col):
    long_short_counts = macd_signals.groupby([date_col,'SIGNAL_DIRECTION'])[code_col].count()
    long_short_counts_df = pd.DataFrame(long_short_counts)
    long_short_counts_df.reset_index(inplace=True)
    long_counts_df = long_short_counts_df[long_short_counts_df['SIGNAL_DIRECTION']==1]
    short_counts_df = long_short_counts_df[long_short_counts_df['SIGNAL_DIRECTION']==-1]
    long_short_counts_df_merged = long_counts_df.merge(short_counts_df, on = date_col)
    long_short_counts_df_merged['long_short_ratio'] = long_short_counts_df_merged['code_x']/(\
                                                                            long_short_counts_df_merged['code_x']+\
                                                                            long_short_counts_df_merged['code_y'])
    long_short_counts_df_merged.index = pd.to_datetime(long_short_counts_df_merged[date_col])
    return long_short_counts_df_merged

In [ ]:
macd_long_short_counts = MACD_long_short_counts(macd_signals, date_col, code_col)

In [ ]:
today = macd_long_short_counts[date_col].values[-1]
macd_long_short_counts[['code_x','code_y']].tail(100).plot(figsize = (18,10));
plt.title("Daily Number of Long Signals vs Short Signals for 50 ETFs by {}".format(today), fontsize = 17);

macd_long_short_counts['long_short_ratio'].tail(100).plot(figsize = (18,10))
long_short_ratio = macd_long_short_counts['long_short_ratio'].values[-1]
plt.title("Long_Short_Ratio: {}, by {}".format(round(long_short_ratio,2), today), fontsize =15);